In [1]:
import pandas as pd
import hashlib
import numpy as np
import random
import itertools
import timeit
!pip install ipython-autotime
%load_ext autotime

time: 0 ns (started: 2024-01-25 16:49:45 +07:00)


In [2]:
df = pd.read_csv('10k-most-common.txt',header= None)

time: 31 ms (started: 2024-01-25 16:49:48 +07:00)


In [3]:
df

,0
0,password
1,123456
2,12345678
3,1234
4,qwerty
...,...
9995,eighty
9996,epson
9997,evangeli
9998,eeeee1


time: 15 ms (started: 2024-01-25 16:49:50 +07:00)


In [4]:
def generate_permutations(word):
    if isinstance(word, str):  
        return [''.join(p) for p in itertools.product(*zip(word.upper(), word.lower()))]
    else:
        return [word]  


df['permutations'] = df[0].apply(generate_permutations)


print(df)

             0                                       permutations
0     password  [PASSWORD, PASSWORd, PASSWOrD, PASSWOrd, PASSW...
1       123456  [123456, 123456, 123456, 123456, 123456, 12345...
2     12345678  [12345678, 12345678, 12345678, 12345678, 12345...
3         1234  [1234, 1234, 1234, 1234, 1234, 1234, 1234, 123...
4       qwerty  [QWERTY, QWERTy, QWERtY, QWERty, QWErTY, QWErT...
...        ...                                                ...
9995    eighty  [EIGHTY, EIGHTy, EIGHtY, EIGHty, EIGhTY, EIGhT...
9996     epson  [EPSON, EPSOn, EPSoN, EPSon, EPsON, EPsOn, EPs...
9997  evangeli  [EVANGELI, EVANGELi, EVANGElI, EVANGEli, EVANG...
9998    eeeee1  [EEEEE1, EEEEE1, EEEEe1, EEEEe1, EEEeE1, EEEeE...
9999    eyphed  [EYPHED, EYPHEd, EYPHeD, EYPHed, EYPhED, EYPhE...

[10000 rows x 2 columns]
time: 359 ms (started: 2024-01-25 16:49:55 +07:00)


In [5]:
df['permutations'] = df[0].apply(generate_permutations)


def hash_permutations(permutations):
    sha1_hashes = [hashlib.sha1(str(p).encode()).hexdigest() for p in permutations]
    md5_hashes = [hashlib.md5(str(p).encode()).hexdigest() for p in permutations]
    return sha1_hashes, md5_hashes


df['sha1_hashes'], df['md5_hashes'] = zip(*df['permutations'].map(hash_permutations))


print(df[['permutations', 'sha1_hashes', 'md5_hashes']])

                                           permutations  \
0     [PASSWORD, PASSWORd, PASSWOrD, PASSWOrd, PASSW...   
1     [123456, 123456, 123456, 123456, 123456, 12345...   
2     [12345678, 12345678, 12345678, 12345678, 12345...   
3     [1234, 1234, 1234, 1234, 1234, 1234, 1234, 123...   
4     [QWERTY, QWERTy, QWERtY, QWERty, QWErTY, QWErT...   
...                                                 ...   
9995  [EIGHTY, EIGHTy, EIGHtY, EIGHty, EIGhTY, EIGhT...   
9996  [EPSON, EPSOn, EPSoN, EPSon, EPsON, EPsOn, EPs...   
9997  [EVANGELI, EVANGELi, EVANGElI, EVANGEli, EVANG...   
9998  [EEEEE1, EEEEE1, EEEEe1, EEEEe1, EEEeE1, EEEeE...   
9999  [EYPHED, EYPHEd, EYPHeD, EYPHed, EYPhED, EYPhE...   

                                            sha1_hashes  \
0     [112bb791304791ddcf692e29fd5cf149b35fea37, 240...   
1     [7c4a8d09ca3762af61e59520943dc26494f8941b, 7c4...   
2     [7c222fb2927d828af22f592134e8932480637c0d, 7c2...   
3     [7110eda4d09e062aa5e4a390b0a572ac0d2c0220, 711...

In [6]:
search_result = df[df['sha1_hashes'].apply(lambda x: 'd54cc1fe76f5186380a0939d2fc1723c44e8a5f7' in x) |
                  df['md5_hashes'].apply(lambda x: 'd54cc1fe76f5186380a0939d2fc1723c44e8a5f7' in x)]


print(search_result)

             0                                       permutations  \
1157  thailand  [THAILAND, THAILANd, THAILAnD, THAILAnd, THAIL...   

                                            sha1_hashes  \
1157  [4869b5e8439d0532a812c0f9aa2de11aa2f61bf1, d55...   

                                             md5_hashes  
1157  [9df9f1156e86609b2ec294b065be043a, 2682f8ab940...  
time: 93 ms (started: 2024-01-25 16:50:04 +07:00)


In [7]:
matching_permutations = []
for word, sha1_hashes, md5_hashes, permutations in zip(search_result[0], search_result['sha1_hashes'], search_result['md5_hashes'], search_result['permutations']):
    if 'd54cc1fe76f5186380a0939d2fc1723c44e8a5f7' in sha1_hashes or 'd54cc1fe76f5186380a0939d2fc1723c44e8a5f7' in md5_hashes:
        matching_permutations.extend(permutations)


matching_words = [word for word in matching_permutations if hashlib.sha1(word.encode()).hexdigest() == 'd54cc1fe76f5186380a0939d2fc1723c44e8a5f7']


print(matching_words)

['ThaiLanD']
time: 0 ns (started: 2024-01-25 16:50:08 +07:00)


In [8]:
def hash_permutations2(permutations):
    sha1_hashes2 = [hashlib.sha1(str(p).encode()).hexdigest() for p in permutations]
    return sha1_hashes2
execution_time = timeit.timeit(lambda: hash_permutations2(df['permutations'][0][0]), number=1)
print(f"Execution time for hashing 1 permutations: {execution_time} seconds")


Execution time for hashing 1 permutations: 0.00023090001195669174 seconds
time: 16 ms (started: 2024-01-25 16:50:11 +07:00)
